# Crawling

'''
여기에 설명 적을거임
'''

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
cd /content/drive/MyDrive/Todays_Chatbot/Crawling

## Import Library

In [ ]:
import numpy as np
import pandas as pd

import json
import requests
from bs4 import BeautifulSoup
from datetime import datetime
from pprint import pprint
from tqdm import tqdm
from tqdm import tqdm_notebook

## 1) 카테고리 정보 크롤링

In [ ]:
def create_category():
  cate_link = []

  url = 'https://ohou.se/store/'

  # 해당 웹 페이지에서 데이터 받아오기
  response = requests.get(url) 
  if response.status_code != requests.codes.ok:
    print("접속 실패")
  else:
    print("접속 성공")

  # 받아온 데이터를 알아볼 수 있는 형태로 파싱(해석)
  soup = BeautifulSoup(response.text, 'html.parser') 

  # 카테고리 부분
  cate = soup.select('#store-index > section.container.store-index-section.store-index-category-list > div > div.category-list.fold > div > a')
  
  for ca in cate:
    cate_link.append([ca.div.div.text, ca['href']])  # 카테고리명, 해당 링크

  # http 부분 추가 & order=review로 바꾸고, affect_type & affect_id 없애는 작업 진행
  for ca in cate_link:
    ca[1] = 'https://ohou.se' + ca[1].replace('popular', 'review').split('&affect')[0]

  return cate_link

In [ ]:
cate_link = create_category()

In [ ]:
cate_link

In [ ]:
# 내보내기
np.save('data/cate_link', cate_link, allow_pickle=True)

### 카테고리 id 추출

In [ ]:
c_num=[]
for i in cate_link:
  c_num.append([i[0], i[1].split('category=')[1].split('&')[0]])

In [ ]:
# 카테고리 아이디
c_num

In [ ]:
category_table = pd.DataFrame(c_num, columns=['category_name','category_id'])
category_table

In [ ]:
category_table.to_csv('data/category_table.csv', encoding='cp949', index=False)

## 2) 상품 정보 크롤링

In [ ]:
def create_product_crawling(category_id):
  product = []
  for cate_id in category_id:
    for page_num in range(1,2): 
      url='https://ohou.se/store/category.json?v=2&category={}&order=review&page={}&per=24'.format(cate_id, page_num)
      resp=requests.get(url)
      data = resp.json()
      for i in range(0,10):
        product.append([cate_id, data['productions'][i]['id'], data['productions'][i]['name']])
  return product

In [ ]:
product = create_product_crawling(category_table['category_id'])

In [ ]:
len(product)

In [ ]:
tmp_table = pd.DataFrame(product, columns=['category_id','product_id','product_name'])

In [ ]:
id_table = tmp_table[['category_id','product_id']]
id_table

In [ ]:
product_table = tmp_table[['product_name','product_id']]
product_table

In [ ]:
# id 중복 제거 (한 상품이 여러 카테고리에 존재하는 경우)
product_table = product_table.drop_duplicates().reset_index(drop=True)
product_table

In [ ]:
product_table.to_csv('data/product_table.csv', encoding='cp949', index=False)
id_table.to_csv('data/id_table.csv', encoding='cp949', index=False)

## 3) 문의글 크롤링

In [ ]:
url='https://ohou.se/production_questions/get_questions.json?product_id=209749&page=2&per=5&v=2'
resp=requests.get(url)
data=resp.json()

In [ ]:
# 제품ID, 유저ID, 구매여부, 질문, 질문시간, 답변, 답변시간
print(data['questions'][4]['production']['id'])
print(data['questions'][4]['production']['name'])
print(data['questions'][4]['production']['explain'])
print('\n')
print(data['questions'][4]['id'])
print(data['questions'][4]['type'])
print(data['questions'][4]['is_buyer'])
print(data['questions'][4]['is_secret'])
print('\n')
print(data['questions'][4]['question'])
print(data['questions'][4]['question_at'])
print('\n')
print(data['questions'][4]['answer']['answer'])
print(data['questions'][4]['answer']['answer_at'])

In [ ]:
def get_question(product_table):
    qna = []
    for prod_id in tqdm_notebook(product_table['product_id']):
        page=1
        while(1):
            url='https://ohou.se/production_questions/get_questions.json?product_id={}&page={}&per=5&v=2'.format(prod_id, page)
            resp=requests.get(url)
            data=resp.json()
            if len(data['questions'])==0: break
            elif data['questions'] is None: break
            else:
                for i in range(0,len(data['questions'])):
                    # 제품ID, 제품명, 옵션
                    prod_id = data['questions'][i]['production']['id']
                    prod_name = data['questions'][i]['production']['name']
                    prod_opt = data['questions'][i]['production']['explain']
                    # 유저ID, 구매여부
                    user_id = data['questions'][i]['id']
                    user_buyer = data['questions'][i]['is_buyer']
                    # 비밀글 여부, 문의 카테고리(label)
                    is_secret = data['questions'][i]['is_secret']
                    label = data['questions'][i]['type']
                    # 질문, 작성시간
                    que = data['questions'][i]['question']
                    que_time = data['questions'][i]['question_at']
                    # 답변, 작성시간
                    if data['questions'][i]['answer'] is not None:
                        ans = data['questions'][i]['answer']['answer']
                        ans_time = data['questions'][i]['answer']['answer_at']
                    else:
                        ans = 'None'
                        ans_time = 'None'
                    # qna list에 추가
                    qna.append([prod_id, prod_name, prod_opt, user_id, user_buyer, is_secret, label, que, que_time, ans, ans_time])
            page = page + 1
    return qna

In [ ]:
product_table = pd.read_csv('data/product_table.csv', encoding='cp949')

In [ ]:
question = get_question(product_table)

In [ ]:
len(question)

In [ ]:
question[0]

In [ ]:
question_table = pd.DataFrame(question, columns=['product_id','product_name','product_option',
                                                 'user_id','user_buyer',
                                                 'is_secret', 'label',
                                                 'question','question_time',
                                                 'answer','answer_time'])

In [ ]:
question_table

In [ ]:
question_table.to_csv('data/question_table.csv', index=False)

#### 예전 코드

In [ ]:
def get_question(id_table):
  qna = []
  for prod_id in tqdm(id_table['product_id']):
    page=1
    while(1):
      url='https://ohou.se/production_questions/get_questions.json?product_id={}&page={}&per=5&v=2'.format(prod_id, page)
      resp=requests.get(url)
      data=resp.json()
      if len(data['questions'])==0:
        break
      elif data['questions'] is None:
        break
      else:
        for i in range(0,len(data['questions'])):
          if data['questions'][i]['answer'] is not None:
            qna.append([data['questions'][i]['question'], data['questions'][i]['answer']])
          else:
            qna.append([data['questions'][i]['question'], 'None'])
      page = page + 1
  return qna

In [ ]:
len(que)

In [ ]:
np.save("data/test_que.npy", que)

## 4) 리뷰 크롤링

In [ ]:
product_table = pd.read_csv('data/product_table.csv', encoding='cp949')

In [ ]:
url='https://ohou.se/production_reviews.json?production_id=209749&page=1&order=best&photo_review_only='
resp=requests.get(url)
data=resp.json()

In [ ]:
data['reviews'][0]

In [ ]:
print(data['reviews'][0]['created_at'])
print(data['reviews'][0]['writer_id'])
print(data['reviews'][0]['praise_count'])
print('\n')
print(data['reviews'][0]['production_information']['id'])
print(data['reviews'][0]['production_information']['name'])
print(data['reviews'][0]['production_information']['explain'])
print('\n')
print(data['reviews'][0]['review']['comment'])
print('\n')
print(data['reviews'][0]['card']['image_url'])

In [ ]:
def get_review(prod_id):
  review = []
  page=1
  while(1):
    url='https://ohou.se/production_reviews.json?production_id={}&page={}&order=best&photo_review_only='.format(prod_id,page)
    resp=requests.get(url)
    data=resp.json()
    if data['reviews'] == []: #is None
      break
    else:
      for i in range(0,len(data['reviews'])):
        review.append(
            [data['reviews'][i]['created_at'],
            data['reviews'][i]['writer_id'],
            data['reviews'][i]['praise_count'],
            
            data['reviews'][i]['production_information']['id'],
            data['reviews'][i]['production_information']['name'],
            data['reviews'][i]['production_information']['explain'],
            
            data['reviews'][i]['review']['comment'],
            
            data['reviews'][i]['card']['image_url']])
    page=page+1
  return review

In [ ]:
product_table['product_id']

In [ ]:
for prod_id in tqdm_notebook(product_table['product_id'][26:30]):
  rev = get_review(prod_id)
  np.save("data/review/review_"+str(prod_id)+".npy", rev)

#### 예전 코드

In [ ]:
def get_review(prod_id):
  review = []
  page=1
  while(1):
    url='https://ohou.se/production_reviews.json?production_id={}&page={}&order=best&photo_review_only='.format(prod_id,page)
    resp=requests.get(url)
    data=resp.json()
    if data['reviews'] == []: #is None
      break
    else:
      for i in range(0,len(data['reviews'])):
        review.append([data['reviews'][i]['card']['thumbnail'],
            data['reviews'][i]['praise_count'],
            data['reviews'][i]['production_information'],
            data['reviews'][i]['review']['comment']])
    page=page+1
  return review

In [ ]:
for prod_id in tqdm(id_table['product_id'][55:60]):
  rev = get_review(prod_id)
  np.save("data/review/review_"+str(prod_id)+".npy", rev)

In [ ]:
np.save("data/review/review_"+str(prod_id)+".npy", rev)